# Solution Model 1
### Faster R-CNN ResNet152 V1 640x640

In [1]:
!pip install tensorflow_io sagemaker -U

  Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
  Using cached s3transfer-0.10.0-py3-none-any.whl.metadata (1.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 409.7 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 348.4 kB/s eta 0:00:000:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.6/147.6 kB 153.7 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 187.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 195.7 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 74.4 kB/s eta 0:00:00 0:00:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 210.4 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 500.7 kB/s eta 0:00:00a 0:00:01
Using cached s3transfer-0.10.0-py3-none-any.whl (82 kB)
  Attempting uninstall: referencing
    Found existing installation: referencing 0.30.2
    Uninstalling 

In [2]:
import os
import sagemaker
from framework import CustomFramework

In [ ]:
role = sagemaker.get_execution_role()
print(role)

In [3]:
inputs = {
    "train": "s3://cd2688-object-detection-tf2/train/",
    "val": "s3://cd2688-object-detection-tf2/val/"
}

tensorboard_s3_prefix = "s3://object-detection-project-singhsegv/faster-rcnn-resnet152/logs/"

In [4]:
%%bash

git clone https://github.com/tensorflow/models.git docker/models

cp docker/models/research/object_detection/exporter_main_v2.py source_dir
cp docker/models/research/object_detection/model_main_tf2.py source_dir

Cloning into 'docker/models'...


Process is interrupted.


In [ ]:
image_name = "tf2-object-detection-faster-rcnn-resnet152"
!sh ./docker/build_and_push.sh $image_name

In [ ]:
with open(os.path.join("docker", "ecr_image_fullname.txt"), "r") as f:
    container = f.readlines()[0][:-1]

print(container)

## TODO: Check me locally

In [ ]:
%%bash

mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/faster_rcnn.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/faster_rcnn_resnet152_v1_640x640_coco17_tpu-8.tar.gz
tar -zxvf /tmp/faster_rcnn.tar.gz --strip-components 2 --directory source_dir/checkpoint faster_rcnn_resnet152_v1_640x640_coco17_tpu-8/checkpoint

In [ ]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path="/opt/training/"
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point="run_training.sh",
    source_dir="source_dir",
    hyperparameters={
        "model_dir": "/opt/training",
        "pipeline_config_path": "faster_rcnn_pipeline.config",
        "num_train_steps": "2000",
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type="ml.g5.xlarge",
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name="tf2-object-detection-faster-rcnn-resnet152"
)

estimator.fit(inputs)